In [2]:
import pandas as pd
import numpy as np
import datetime

In [35]:
train = pd.read_csv("./Data/train/train.csv")
train["PurchaseMonth"] = (pd.to_datetime(train["OrderDate"],format = "%d/%m/%y")).dt.month
productAttrs= pd.read_csv("./Data/train/product_attributes.csv")
test = pd.read_csv("./Data/test.csv")

In [151]:
#list(set(train["productid"]) - set(productAttrs["productid"]))
train.shape

(132551, 5)

In [ ]:
12947666,5
 12995896,5
 13017336,6
 13072176,8
 13072610,8
 13653544,8
 13773574,
 14101712,
 14122346,
 14122348,
 14129477

In [145]:
productMatrix[productMatrix["productid"] == 11147300]

,productid,Category,Collection,Color,Fabric,Fit,Material,Neckline,Season,Sleeve Length,BinaryRep,TargetCode
57,11147300,1.0,0.0,1.0,1.0,1.0,0.0,1.0,1.0,1.0,101110111,8


In [140]:
#-----------There are more than 1 purchases
aggData = train[train["UserId"]!=26784].groupby(["UserId","productid"])["Quantity"].count().reset_index()
aggData.columns = ["UserId","productid","PurchaseCount"]
#aggData.sort_values(by = "PurchaseCount", ascending=False)

In [132]:
#(productAttrs.groupby(["productid","attribute_name"]).count().reset_index())
len(np.unique(productMatrix["productid"]))

3015

In [128]:
productMatrix = (productAttrs.groupby(["productid","attribute_name"]).count().reset_index())
productMatrix["attributevalue"] = (productMatrix["attributevalue"] >= 1).astype("int")

productMatrix = (productMatrix
                 .pivot(index = "productid",columns = "attribute_name").fillna(0)
                ).reset_index()

productMatrix["BinaryRep"] = productMatrix[productMatrix.columns[1:]].apply(
    lambda x: ''.join(x.astype(int).astype(str)),
    axis=1
)

productMatrix.columns = [col[1] if (len(col[1]) > 0) else col[0] for col in productMatrix.columns]

#-------Merge categories with less that 10 count products
targetCodesWithLessProducts = (productMatrix.groupby("BinaryRep")["productid"]
                               .count().sort_values( ascending = False).reset_index()
                              )
targetCodesWithLessProducts = targetCodesWithLessProducts[targetCodesWithLessProducts["productid"] < 10][["BinaryRep"]]

productMatrix = productMatrix.merge(targetCodesWithLessProducts, how="left", on= ["BinaryRep"],indicator=True)
productMatrix.loc[tmp["_merge"] == "both","BinaryRep"] = "000000000"
productMatrix["TargetCode"] = productMatrix["BinaryRep"].astype("category").cat.codes
productMatrix.drop("_merge", axis = 1, inplace = True)

productMatrix.head(10)
#productMatrix.columns[1:]

,productid,Category,Collection,Color,Fabric,Fit,Material,Neckline,Season,Sleeve Length,BinaryRep,TargetCode
0,11139192,1.0,0.0,1.0,1.0,1.0,0.0,0.0,1.0,0.0,101110010,6
1,11139194,1.0,0.0,1.0,1.0,1.0,0.0,0.0,1.0,0.0,101110010,6
2,11139524,1.0,0.0,1.0,1.0,1.0,0.0,1.0,1.0,1.0,101110111,8
3,11139560,1.0,0.0,1.0,1.0,1.0,0.0,1.0,1.0,1.0,101110111,8
4,11139588,1.0,0.0,1.0,1.0,1.0,0.0,0.0,1.0,0.0,101110010,6
5,11139650,1.0,0.0,1.0,1.0,1.0,0.0,1.0,1.0,1.0,101110111,8
6,11141306,1.0,0.0,1.0,1.0,0.0,0.0,0.0,1.0,0.0,101100010,5
7,11141308,1.0,0.0,1.0,1.0,1.0,0.0,0.0,1.0,0.0,101110010,6
8,11141318,1.0,0.0,1.0,1.0,1.0,0.0,0.0,1.0,0.0,101110010,6
9,11141320,1.0,0.0,1.0,1.0,1.0,0.0,0.0,1.0,0.0,101110010,6


In [146]:
productMatrix.groupby(["TargetCode"]).count()

,productid,Category,Collection,Color,Fabric,Fit,Material,Neckline,Season,Sleeve Length,BinaryRep
TargetCode,,,,,,,,,,,
0,64,64,64,64,64,64,64,64,64,64,64
1,19,19,19,19,19,19,19,19,19,19,19
2,11,11,11,11,11,11,11,11,11,11,11
3,19,19,19,19,19,19,19,19,19,19,19
4,221,221,221,221,221,221,221,221,221,221,221
5,359,359,359,359,359,359,359,359,359,359,359
6,163,163,163,163,163,163,163,163,163,163,163
7,11,11,11,11,11,11,11,11,11,11,11
8,1927,1927,1927,1927,1927,1927,1927,1927,1927,1927,1927


In [129]:
cols = list(productMatrix.columns[1:])
productMatrix[cols].groupby(cols).count()

Empty DataFrame
Columns: []
Index: [(0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 1.0, 000000000, 0), (0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 1.0, 0.0, 000000000, 0), (0.0, 0.0, 0.0, 1.0, 0.0, 0.0, 0.0, 0.0, 0.0, 000000000, 0), (0.0, 0.0, 0.0, 1.0, 0.0, 0.0, 1.0, 0.0, 0.0, 000000000, 0), (0.0, 0.0, 0.0, 1.0, 1.0, 0.0, 0.0, 0.0, 0.0, 000000000, 0), (0.0, 0.0, 0.0, 1.0, 1.0, 0.0, 0.0, 1.0, 0.0, 000000000, 0), (0.0, 0.0, 0.0, 1.0, 1.0, 0.0, 1.0, 0.0, 0.0, 000000000, 0), (0.0, 0.0, 1.0, 0.0, 0.0, 0.0, 0.0, 1.0, 0.0, 000000000, 0), (0.0, 0.0, 1.0, 0.0, 1.0, 0.0, 1.0, 1.0, 1.0, 000000000, 0), (0.0, 0.0, 1.0, 1.0, 0.0, 0.0, 0.0, 1.0, 0.0, 001100010, 1), (0.0, 0.0, 1.0, 1.0, 0.0, 0.0, 1.0, 1.0, 0.0, 000000000, 0), (0.0, 0.0, 1.0, 1.0, 1.0, 0.0, 0.0, 1.0, 0.0, 000000000, 0), (0.0, 0.0, 1.0, 1.0, 1.0, 0.0, 0.0, 1.0, 1.0, 000000000, 0), (0.0, 0.0, 1.0, 1.0, 1.0, 0.0, 1.0, 0.0, 1.0, 000000000, 0), (0.0, 0.0, 1.0, 1.0, 1.0, 0.0, 1.0, 1.0, 1.0, 001110111, 2), (1.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 1.0, 0.0, 000000000, 0), (1.0, 0.0, 0.0, 0.0, 1.0, 0.0, 0.0, 0.0, 0.0, 000000000, 0), (1.0, 0.0, 0.0, 0.0, 1.0, 0.0, 0.0, 1.0, 0.0, 000000000, 0), (1.0, 0.0, 0.0, 0.0, 1.0, 0.0, 1.0, 0.0, 1.0, 000000000, 0), (1.0, 0.0, 0.0, 0.0, 1.0, 0.0, 1.0, 1.0, 1.0, 000000000, 0), (1.0, 0.0, 0.0, 1.0, 0.0, 0.0, 0.0, 1.0, 0.0, 000000000, 0), (1.0, 0.0, 0.0, 1.0, 0.0, 0.0, 0.0, 1.0, 1.0, 000000000, 0), (1.0, 0.0, 0.0, 1.0, 0.0, 0.0, 1.0, 1.0, 1.0, 000000000, 0), (1.0, 0.0, 0.0, 1.0, 1.0, 0.0, 0.0, 1.0, 0.0, 000000000, 0), (1.0, 0.0, 0.0, 1.0, 1.0, 0.0, 0.0, 1.0, 1.0, 000000000, 0), (1.0, 0.0, 0.0, 1.0, 1.0, 0.0, 1.0, 1.0, 1.0, 000000000, 0), (1.0, 0.0, 1.0, 0.0, 0.0, 0.0, 0.0, 1.0, 0.0, 101000010, 3), (1.0, 0.0, 1.0, 0.0, 0.0, 1.0, 0.0, 0.0, 0.0, 000000000, 0), (1.0, 0.0, 1.0, 0.0, 0.0, 1.0, 0.0, 1.0, 0.0, 101001010, 4), (1.0, 0.0, 1.0, 0.0, 1.0, 0.0, 0.0, 0.0, 0.0, 000000000, 0), (1.0, 0.0, 1.0, 1.0, 0.0, 0.0, 0.0, 0.0, 0.0, 000000000, 0), (1.0, 0.0, 1.0, 1.0, 0.0, 0.0, 0.0, 1.0, 0.0, 101100010, 5), (1.0, 0.0, 1.0, 1.0, 0.0, 0.0, 1.0, 1.0, 1.0, 000000000, 0), (1.0, 0.0, 1.0, 1.0, 1.0, 0.0, 0.0, 0.0, 0.0, 000000000, 0), (1.0, 0.0, 1.0, 1.0, 1.0, 0.0, 0.0, 1.0, 0.0, 101110010, 6), (1.0, 0.0, 1.0, 1.0, 1.0, 0.0, 0.0, 1.0, 1.0, 101110011, 7), (1.0, 0.0, 1.0, 1.0, 1.0, 0.0, 1.0, 1.0, 0.0, 000000000, 0), (1.0, 0.0, 1.0, 1.0, 1.0, 0.0, 1.0, 1.0, 1.0, 101110111, 8), (1.0, 1.0, 1.0, 1.0, 0.0, 0.0, 0.0, 1.0, 0.0, 111100010, 9), (1.0, 1.0, 1.0, 1.0, 1.0, 0.0, 0.0, 0.0, 0.0, 000000000, 0), (1.0, 1.0, 1.0, 1.0, 1.0, 0.0, 0.0, 1.0, 0.0, 111110010, 10), (1.0, 1.0, 1.0, 1.0, 1.0, 0.0, 1.0, 1.0, 0.0, 000000000, 0), (1.0, 1.0, 1.0, 1.0, 1.0, 0.0, 1.0, 1.0, 1.0, 111110111, 11)]

In [52]:
cols

['Category',
 'Collection',
 'Color',
 'Fabric',
 'Fit',
 'Material',
 'Neckline',
 'Season',
 'Sleeve Length',
 'BinaryRep',
 'TargetCode']

In [150]:
productMatrix1 = productMatrix[["productid","TargetCode"]]
productMatrix1 = pd.concat([productMatrix1
                           ,pd.DataFrame({"productid":list(set(train["productid"]) - set(productAttrs["productid"]))
                                          ,"TargetCode":[8]*11
                                         })
                           ],axis = 0)
                                         
productMatrix1.to_csv("./Data/train/ProductMatrix",index = False)

In [121]:
productMatrix.columns

MultiIndex(levels=[['attributevalue', 'BinaryRep'], ['Category', 'Collection', 'Color', 'Fabric', 'Fit', 'Material', 'Neckline', 'Season', 'Sleeve Length', '']],
           labels=[[0, 0, 0, 0, 0, 0, 0, 0, 0, 1], [0, 1, 2, 3, 4, 5, 6, 7, 8, 9]],
           names=[None, 'attribute_name'])

In [78]:
productAttrs.pivot(index = "productid",columns="attribute_name")

ValueError: Index contains duplicate entries, cannot reshape

In [44]:
train[train["UserId"] ==26784].groupby("OrderDate").count()

,UserId,productid,Quantity
OrderDate,,,
01/04/18,142,142,142
01/05/18,149,149,149
01/06/18,104,104,104
01/07/18,451,451,451
01/08/18,152,152,152
01/09/18,175,175,175
01/10/18,149,149,149
02/04/18,128,128,128
02/05/18,120,120,120


In [65]:
train.head()

,UserId,productid,Quantity,OrderDate
0,18075,12322648,1,01/04/18
1,6820,12371370,1,01/04/18
2,6820,12973004,1,01/04/18
3,6820,12657560,1,01/04/18
4,6820,11659914,1,01/04/18


In [29]:
train

array(['Category', 'Collection', 'Color', 'Fabric', 'Fit', 'Material',
       'Neckline', 'Season', 'Sleeve Length'], dtype=object)